In [1]:
import pandas as pd

In [17]:
eICU_med_path = '/home/qlunlp/ylh/database/eICU/medication.csv'
origin_med_pd = pd.read_csv(eICU_med_path, dtype={'drugname': str})

/tmp/ipykernel_354859/2992554931.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  origin_med_pd = pd.read_csv(eICU_med_path, dtype={'drugname': str})


In [ ]:

med_pd = pd.read_csv(eICU_med_path, usecols=['drugname'], dtype={'drugname': str})
med_pd.head()

,drugname
0,METOPROLOL TARTRATE 25 MG PO TABS
1,3 ML - IPRATROPIUM-ALBUTEROL 0.5-2.5 (3) MG/...
2,ASPIRIN EC 81 MG PO TBEC
3,3 ML - IPRATROPIUM-ALBUTEROL 0.5-2.5 (3) MG/...
4,ENOXAPARIN SODIUM 40 MG/0.4ML SC SOLN


In [6]:
len(med_pd)

7301853

In [7]:
med_pd.drop(med_pd[med_pd['drugname'].isnull()].index, inplace=True)
len(med_pd)

4696249

In [8]:
med_pd.drop_duplicates(subset=['drugname'], keep='first', inplace=True)
len(med_pd)

1411

In [12]:
med_pd = med_pd.reset_index(drop=True)
len(med_pd)

1411

In [14]:
med_pd.to_csv('/home/qlunlp/ylh/database/tmp/unique_drugname.csv', index=False)

In [16]:
drugname = pd.read_csv('/home/qlunlp/ylh/database/tmp/unique_drugname.csv', dtype={'drugname': str})
drugname.head()

,drugname
0,METOPROLOL TARTRATE 25 MG PO TABS
1,3 ML - IPRATROPIUM-ALBUTEROL 0.5-2.5 (3) MG/...
2,ASPIRIN EC 81 MG PO TBEC
3,ENOXAPARIN SODIUM 40 MG/0.4ML SC SOLN
4,HYDROCORTISONE NA SUCCINATE PF 100 MG IJ SOLR


In [ ]:
for item in drugname['drugname']:
    cnt = len(origin_med_pd[origin_med_pd['drugname'] == item])
    if cnt == 0:
        print(f"Drugname {item} not found in original medication data.")    

## 映射文件

In [33]:
rxnorm_mappings_path = "/home/qlunlp/ylh/database/drug/rxnorm_mappings.txt"
origin_rxnorm_mappings = pd.read_csv(rxnorm_mappings_path, sep="|", dtype={'RXSTRING': str, 'RXCUI': str})
origin_rxnorm_mappings.head()

,SETID,SPL_VERSION,RXCUI,RXSTRING,RXTTY
0,00002127-02bc-4c66-b0c3-ca29d8224afc,1,108204,BETADINE 5 % Topical Spray,PSN
1,00002127-02bc-4c66-b0c3-ca29d8224afc,1,108204,Betadine 5 % Topical Spray,SY
2,00002127-02bc-4c66-b0c3-ca29d8224afc,1,108204,Betadine 50 MG/ML Topical Spray,SY
3,00002127-02bc-4c66-b0c3-ca29d8224afc,1,108204,povidone-iodine 50 MG/ML Topical Spray [Betadine],SBD
4,00002127-02bc-4c66-b0c3-ca29d8224afc,1,238850,povidone-iodine 5 % Topical Spray,PSN


In [34]:
rxnorm_mapping = origin_rxnorm_mappings[['RXCUI', 'RXSTRING']]
len(rxnorm_mapping)

459178

In [36]:
rxnorm_mapping.dropna(inplace=True)
rxnorm_mapping.drop_duplicates(inplace=True)
len(rxnorm_mapping)

/tmp/ipykernel_354859/1574281697.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rxnorm_mapping.dropna(inplace=True)
/tmp/ipykernel_354859/1574281697.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rxnorm_mapping.drop_duplicates(inplace=True)


66558

In [38]:
rxnorm_mapping.head()

,RXCUI,RXSTRING
0,108204,BETADINE 5 % Topical Spray
1,108204,Betadine 5 % Topical Spray
2,108204,Betadine 50 MG/ML Topical Spray
3,108204,povidone-iodine 50 MG/ML Topical Spray [Betadine]
4,238850,povidone-iodine 5 % Topical Spray


In [28]:
import re
def normalize_text(s: str) -> str:
    """基础规范化：小写，去多余空格，标准化常见缩写等。"""
    s = s.lower()
    # 去掉多余连字符，两边留空格方便匹配
    s = s.replace('-', ' ')
    s = s.replace('/', ' / ')
    # 一些常见缩写展开
    s = s.replace(' ipratropium br ', ' ipratropium bromide ')
    s = s.replace(' tabs ', ' tablet ')
    s = s.replace(' tbec ', ' tablet enteric ')
    s = s.replace(' inj ', ' injection ')
    s = re.sub(r'\s+', ' ', s)
    return s.strip()

In [42]:
rxnorm_mapping['RXSTRING'] = rxnorm_mapping['RXSTRING'].astype(str).apply(normalize_text)
rxnorm_mapping.drop_duplicates(subset=['RXSTRING'], keep='first', inplace=True)
rxnorm_mapping = rxnorm_mapping.reset_index(drop=True)
rxnorm_mapping

/tmp/ipykernel_354859/273352972.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rxnorm_mapping['RXSTRING'] = rxnorm_mapping['RXSTRING'].astype(str).apply(normalize_text)
/tmp/ipykernel_354859/273352972.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rxnorm_mapping.drop_duplicates(subset=['RXSTRING'], keep='first', inplace=True)


,RXCUI,RXSTRING
0,108204,betadine 5 % topical spray
1,108204,betadine 50 mg / ml topical spray
2,108204,povidone iodine 50 mg / ml topical spray [beta...
3,238850,povidone iodine 5 % topical spray
4,238850,povidone iodine 50 mg / ml topical spray
...,...,...
63637,763089,plan b pack
63638,763089,{2 (levonorgestrel 0.75 mg oral tablet) } pack...
63639,209011,phoslo 667 mg (ca 169 mg) oral tablet
63640,209011,phoslo 667 mg oral tablet


In [41]:
len(rxnorm_mapping)

63642

In [43]:
rxnorm_mapping.to_csv('/home/qlunlp/ylh/database/tmp/processed_rxnorm_mappings.csv', index=False)